In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from model import Model, CustomDataset
from utils import rgb_to_gray
import matplotlib.pyplot as plt
import gymnasium as gym
import numpy as np
import imageio
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [2]:
class PongAgent:
    def __init__(self, hidden_conv_layer_dims, hidden_lin_layer_dims, game = "ALE/Pong-v5", epsilon = 0.5, gamma=0.99, epochs=25, num_actions=6) -> None:
        self.target_network = Model(hidden_conv_layer_dims, hidden_lin_layer_dims)
        self.target_network._save__model__('0')
        self.online_network = torch.load("epoch_0_model.pth")
        self.gamma = gamma
        self.epsilon = epsilon
        self.epochs = epochs
        self.decay = pow(100*self.epsilon, 1/self.epochs)
        self.num_skip_frame = 4
        self.num_actions = num_actions
        self.game = game
        self.epochs = epochs
    def evaluate(self, iter) -> None:
        env = gym.make("ALE/Pong-v5", render_mode="rgb_array")
        for ____ in range(1):
            obs, _ = env.reset()
            images = []
            reward = 0
            action = 0
            state = [rgb_to_gray(obs)]
            for __ in range(self.num_skip_frame-1):
                obs, rew, done, info, _ = env.step(action=action)
                images.append(env.render())
                if done:
                    break
                reward += rew
                obs = rgb_to_gray(obs)
                state.append(obs)
            if len(state) != 4:
                print("F")
                continue
            while not done:
                if state is None:
                    break
                Q_s = self.online_network(state).detach()
                action = torch.argmax(Q_s).detach().item()
                obs, rew, done, info, _ = env.step(action=action)
                images.append(env.render())
                reward += rew * (1-done)
                state = state[:-1].append(rgb_to_gray(obs))
            print(len(images))
            imageio.mimsave(f"./gifs/iter_{iter}_{____+1}.gif", images, duration=1)
        print(f"Average reward after {iter} iterations : {reward/5}")
        return reward/5
    
    def choose_action(self, Q_s) -> int:
        if np.random.random()<self.epsilon:
            return np.random.randint(0,self.num_actions)
        else:
            if not isinstance(Q_s, torch.Tensor):
                Q_s = torch.tensor(Q_s)
            return torch.argmax(Q_s).item()
        
    def _generate_training_data(self):
        replay_buffer_data = []
        replay_buffer_labels = []
        env = gym.make(self.game)
        num_episodes = 1
        for episode_no in tqdm(range(num_episodes), desc=f"Simulating episodes "):
            rewards = []
            obs, _ = env.reset()
            done = False
            action = 0
            while not done:
                reward = 0
                state = []
                for __ in range(self.num_skip_frame):
                    obs, rew, done, info, _ = env.step(action=action)
                    if done:
                        break
                    reward += rew
                    obs = rgb_to_gray(obs)
                    state.append(obs)
                if len(state) != 4:
                    break
                state = np.array(state)
                rewards.append(reward)
                Q_s = self.online_network(state[np.newaxis, :]).detach()
                action = self.choose_action(Q_s)
                replay_buffer_data.append((state, action))
            for i in range(len(rewards)-2, -1, -1):
                rewards[i] += self.gamma * rewards[i+1]
            replay_buffer_labels.extend(rewards)
        return replay_buffer_data, replay_buffer_labels
    
    def _train_agent(self):
        rewards = []
        for epoch in range(self.epochs):
            for __ in range(3):
                replay_buffer_data, replay_buffer_labels = self._generate_training_data()
                train_dataset = CustomDataset(replay_buffer_data, replay_buffer_labels)
                self.online_network._train__instance__(train_dataset=train_dataset)
            self.online_network._save__model__(str(epoch+1))
            del self.target_network
            del replay_buffer_data
            del replay_buffer_labels
            self.target_network = torch.load(f"epoch_{epoch+1}_model.pth")
            print("Model Updated and saved!")
            print(self.evaluate(epoch+1))
            # rewards.append(self.evaluate(epoch+1))
            self.epsilon = self.epsilon / self.decay
        # plt.plot(rewards)
        # plt.show()

In [3]:
hidden_conv_layer_dims = [(4, 16, 8, 3, 0), (16, 32, 5, 3, 2), (32, 64, 3, 2, 0)]
hidden_lin_layer_dims = [(1024, 256), (256, 32), (32, 6)]
agent = PongAgent(hidden_conv_layer_dims, hidden_lin_layer_dims, game="ALE/Pong-v5", gamma=0.99, epsilon=0.5, epochs=25, num_actions=6)
agent._train_agent()

A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]
 20%|██        | 1/5 [00:00<00:01,  3.11it/s]

Epoch 1/5, Loss: 2015.3626299417851


 40%|████      | 2/5 [00:00<00:01,  2.63it/s]

Epoch 2/5, Loss: 1794.6113699496127


 60%|██████    | 3/5 [00:01<00:00,  2.47it/s]

...

Epoch 3/5, Loss: 15918.143839012231


 80%|████████  | 4/5 [00:01<00:00,  2.68it/s]

Epoch 4/5, Loss: 12778.356423061185


 80%|████████  | 4/5 [00:01<00:00,  2.36it/s]


KeyboardInterrupt: 

In [ ]:
x = torch.tensor([[1, 2, 3, 4, 5, 6], [7, 8, 9, 10, 11, 12], [13, 14, 15, 16, 17, 18]])
ac = torch.tensor([0, 2])
ac.unsqueeze(1)
print(x)
y = x.gather(1, ac.unsqueeze(1))
print(len(y.shape))

tensor([[ 1,  2,  3,  4,  5,  6],
        [ 7,  8,  9, 10, 11, 12],
        [13, 14, 15, 16, 17, 18]])
2
